# Building a basic domain specific question and answer chat application(RAG)

## Notebook content
This notebook contains the steps and code to demonstrate Retrieval Augumented Generation to build a chat application to answer questions specific to a domain. It introduces commands for data retrieval, knowledge base building & querying, and model testing.

### About Retrieval Augmented Generation
Retrieval Augmented Generation (RAG) is a versatile pattern that can unlock a number of use cases requiring factual recall of information, such as querying a knowledge base in natural language.

In its simplest form, RAG requires 3 steps:

- Index knowledge base passages (once)
- Retrieve relevant passage(s) from knowledge base (for every user query)
- Generate a response by feeding retrieved passage into a large language model (for every user query)

## Contents

This notebook contains the following parts:

- [Introduction to RAG](#intro)
- [Setup](#setup)
- [Document data loading](#data)
- [Build up knowledge base](#build_base)
- [Foundation Models on watsonx](#models)
- [Generate a retrieval-augmented response to a question](#predict)
- [References](#references)


<a id="intro"></a>
## Introduction to Retrieval Augmented Generation(RAG)

RAG implementation flow:
![image](https://dataplatform.cloud.ibm.com/docs/api/content/wsj/analyze-data/images/fm-rag-embed.svg?context=wx&locale=en)

<a id="setup"></a>
##  Set up the environment

- Create API key from platform.openai.com


### Install and import the dependecies

In [ ]:
!pip install "langchain==0.1.10" | tail -n 1
!pip install "ibm-watsonx-ai>=0.2.6" | tail -n 1
!pip install -U langchain_ibm | tail -n 1
!pip install wget | tail -n 1
!pip install sentence-transformers | tail -n 1
!pip install "chromadb==0.3.26" | tail -n 1
!pip install "pydantic==1.10.0" | tail -n 1
!pip install "sqlalchemy==2.0.1" | tail -n 1
!pip install "pypdf" | tail -n 1
!pip install "openai" | tail -n 1
!pip install "tiktoken" | tail -n 1

In [1]:
import os, getpass

### OpenAI API connection
This cell defines the credentials required to work with Model inferencing.


In [17]:
openai_apikey = ""

<a id="data"></a>
## Document data loading

Download the file with State of the Union.

In [4]:
import requests
import os

filename = 'itr-faq.pdf'
url = 'https://www.incometax.gov.in/iec/foportal/sites/default/files/2024-06/Top%2010%20issues%20of%20taxpayers%20updated.pdf'

if not os.path.isfile(filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded {filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")


Downloaded itr-faq.pdf


In [5]:
ls -ltr itr-faq*

-rw-r--r--  1 krishna  staff  231539 Jul 27 08:05 itr-faq.pdf


<a id="build_base"></a>
## Build up knowledge base

The most common approach in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.

In this basic example, we take the Income tax returns FAQ content (in PDF file), split it into chunks, embed it using an open-source embedding model, load it into <a href="https://www.trychroma.com/" target="_blank" rel="noopener no referrer">Chroma</a>, and then query it.

In [6]:
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

loader = PyPDFLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

The dataset we are using is already split into self-contained passages that can be ingested by Chroma.

### Create an embedding function

Note that you can feed a custom embedding function to be used by chromadb. The performance of Chroma db may differ depending on the embedding model used. In following example we use watsonx.ai Embedding service. We can check available embedding models using `get_embedding_model_specs`

In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI

os.environ["OPENAI_API_KEY"] = openai_apikey

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

docsearch = Chroma.from_documents(texts, embeddings, collection_name="itr-docs-openai", persist_directory= "/tmp/vector_data_oai")

### LangChain CustomLLM wrapper for Openai model
Initialize the `OpenAI` class from Langchain

In [12]:
from langchain.llms import OpenAI

llm_openai = OpenAI(temperature=0.7, max_tokens=200)

<a id="predict"></a>
## Generate a retrieval-augmented response to a question

Build the `RetrievalQA` (question answering chain) to automate the RAG task.

In [13]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm_openai, chain_type="stuff", retriever=docsearch.as_retriever())

### Select questions

Get questions from the previously loaded test dataset.

In [14]:
query = "how to select a bank for refund"
print(f"Question:\n  {query}")
answer = qa.invoke(query)["result"]
print(f"Answer:\n {answer}")

Question:
  how to select a bank for refund
Answer:
  To select a bank for refund, the user needs to add a bank account in which they would like to receive the refund. This can be done by going to Profile>> My Bank Account and adding the necessary details. The request will then be sent to the respective bank or NPCI for validation. Once the validation is successful, the taxpayer can nominate the bank account for the refund.


In [15]:
query = "What are the documentes required to register for legal heir"
print(f"Question:\n  {query}")
answer = qa.invoke(query)["result"]
print(f"Answer:\n {answer}")

Question:
  What are the documentes required to register for legal heir
Answer:
 
The documents required to register as a legal heir may vary depending on the specific circumstances, but generally they include:

1. Copy of the deceased person's PAN card.
2. Copy of the legal heir certificate issued by a court of law or local revenue authorities.
3. Scanned copy of the PAN card of the legal heir.
4. Scanned PDF copy of identity and address proof of the legal heir (such as passport, voter ID, driving license, Aadhaar card, or bank passbook with photo).
5. Letter in writing requesting to register as a legal heir.
6. Scanned copy of the death certificate issued by the Municipal Authority, Corporation, or Registrar of Deaths.
7. Copy of any order passed in the name of the deceased, if applicable.
8. Copy of a letter of indemnity (optional).
9. In case any document is in a vernacular language, a Hindi/English translation of the document duly notarized, along with a copy of the


In [16]:
query = "Hello"
print(f"Question:\n  {query}")
answer = qa.invoke(query)["result"]
print(f"Answer:\n {answer}")

Question:
  Hello
Answer:
  I'm sorry, I am only able to assist with questions related to income tax filing. How can I help you with that?


---

<a id="references"></a>
## References
- [Using vectorized text with retrieval-augmented generation tasks](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-embedding-rag.html?context=wx&audience=wdp)

- [Sample notebook](https://dataplatform.cloud.ibm.com/exchange/public/entry/view/d3a5f957-a93b-46cd-82c1-c8d37d4f62c6?context=wx?context=wx&audience=wdp)
